In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./mnist/data/', one_hot=True)#데이터 내려 받고 레이블을 원핫인코딩방식으로 읽는다.

W0716 11:04:14.656659 11332 deprecation.py:323] From <ipython-input-1-2eaef60a95cb>:3: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0716 11:04:14.659649 11332 deprecation.py:323] From c:\python36\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0716 11:04:14.660614 11332 deprecation.py:323] From c:\python36\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement 

Extracting ./mnist/data/train-images-idx3-ubyte.gz


W0716 11:04:14.875097 11332 deprecation.py:323] From c:\python36\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0716 11:04:14.876038 11332 deprecation.py:323] From c:\python36\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0716 11:04:14.919918 11332 deprecation.py:323] From c:\python36\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please 

Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [2]:
X = tf.placeholder(tf.float32, [None,784]) # None은 크기가 정해지지 않았다는 것, 한번에 학습시킬 MNIST 이미지 개수 지정 값
Y = tf.placeholder(tf.float32, [None,10]) # 배치 크기 미리 정해도 되지만 학습 개수 계속 바꿔가면서 실험하려는 경우 None넣으면 알아서 계산

In [3]:
# 신경망 구축
W1 = tf.Variable(tf.random_normal([784, 256], stddev=0.01))
L1 = tf.nn.relu(tf.matmul(X,W1))

W2 = tf.Variable(tf.random_normal([256, 256], stddev=0.01))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L2, W3)

In [4]:
# 손실값 구하고 평균 송실값 구하기, 최적화 수행하도록 그래프 구성
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [5]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [6]:
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

In [7]:
for epoch in range(15):
    total_cost = 0
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        _, cost_val = sess.run([optimizer, cost],
                              feed_dict={X:batch_xs, Y:batch_ys})
        total_cost += cost_val
    print('Epoch:','%04d' % (epoch+1),
         'Avg.cost = ', '{:3f}'.format(total_cost / total_batch))
print('최적화 완료!')

Epoch: 0001 Avg.cost =  0.397060
Epoch: 0002 Avg.cost =  0.145760
Epoch: 0003 Avg.cost =  0.093560
Epoch: 0004 Avg.cost =  0.068741
Epoch: 0005 Avg.cost =  0.050076
Epoch: 0006 Avg.cost =  0.039192
Epoch: 0007 Avg.cost =  0.029875
Epoch: 0008 Avg.cost =  0.026203
Epoch: 0009 Avg.cost =  0.018566
Epoch: 0010 Avg.cost =  0.017269
Epoch: 0011 Avg.cost =  0.016305
Epoch: 0012 Avg.cost =  0.011162
Epoch: 0013 Avg.cost =  0.015106
Epoch: 0014 Avg.cost =  0.009133
Epoch: 0015 Avg.cost =  0.007723
최적화 완료!


In [8]:
is_correct = tf.equal(tf.argmax(model,1), tf.argmax(Y,1))

In [9]:
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [10]:
print('정확도:',sess.run(accuracy,
                     feed_dict={X:mnist.test.images,
                               Y:mnist.test.labels}))

정확도: 0.9813


In [11]:
keep_prob = tf.placeholder(tf.float32) # 학습에는 떼놔도 예측시에는 전부 사용 하도록 설정하기 위한 것

W1 = tf.Variable(tf.random_normal([784, 256], stddev=0.01))
L1 = tf.nn.relu(tf.matmul(X, W1))
L1 = tf.nn.dropout(L1, keep_prob) # dropout 쓰기만 하면 작용, 0.8은 80% 뉴런을 사용하겠다는 것

W2 = tf.Variable(tf.random_normal([256, 256], stddev=0.01))
L2 = tf.nn.relu(tf.matmul(L1,W2))
L2 = tf.nn.dropout(L2, keep_prob)

_, cost_val = sess.run([optimizer, cost], feed_dict={X:batch_xs, Y: batch_ys, keep_prob:0.8}) # 0.8은 80%의 뉴런만 쓰겠다는 것
print('정확도:', sess.run(accuracy, feed_dict={X:mnist.test.images,
                                           Y: mnist.test.labels,
                                           keep_prob:1})) # 예상할때는 전부 사용

W0716 11:04:46.438511 11332 deprecation.py:506] From <ipython-input-11-5c55255b9bf4>:5: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


정확도: 0.982


In [12]:
keep_prob = tf.placeholder(tf.float32) # 학습에는 떼놔도 예측시에는 전부 사용 하도록 설정하기 위한 것

W1 = tf.Variable(tf.random_normal([784, 256], stddev=0.01))
L1 = tf.nn.relu(tf.matmul(X, W1))
L1 = tf.nn.dropout(L1, keep_prob) # dropout 쓰기만 하면 작용, 0.8은 80% 뉴런을 사용하겠다는 것

W2 = tf.Variable(tf.random_normal([256, 256], stddev=0.01))
L2 = tf.nn.relu(tf.matmul(L1,W2))
L2 = tf.nn.dropout(L2, keep_prob)

W3 = tf.Variable(tf.random_normal([256,10], stddev=0.01))
model = tf.matmul(L2,W3)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [13]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for eposh in range(3):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
        _, cost_val = sess.run([optimizer, cost],
                              feed_dict = {X:batch_xs,
                                          Y:batch_ys,
                                          keep_prob:0.8})
        total_cost += cost_val
    print('Epoch:', '%04d' % (epoch+1),
         'Avg. cost =', '{:.3f}'.format(total_cost/total_batch))
print('최적화 완료!')

Epoch: 0015 Avg. cost = 0.421
Epoch: 0015 Avg. cost = 0.164
Epoch: 0015 Avg. cost = 0.114
최적화 완료!


In [14]:
is_correct = tf.equal(tf.argmax(model,1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도:', sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels, keep_prob:1}))

정확도: 0.9712


In [15]:
import matplotlib.pyplot as plt
import numpy as np

In [16]:
labels = sess.run(model, feed_dict={X: mnist.test.images,
                                   Y: mnist.test.labels,
                                   keep_prob: 1})

In [17]:
fig = plt.figure()

<Figure size 432x288 with 0 Axes>

In [18]:
for i in range(10): # 2행 5열 그래프 제작, i+1번째 숫자에 이미지 출력
    subplot = fig.add_subplot(2, 5, i + 1) # 깨끗한 이미지 위해 x,y 눈금 미출력
    subplot.set_xticks([])
    subplot.set_yticks([])
    subplot.set_title('%d' % np.argmax(labels[i])) # 출력 이미지 위 예측 숫자 출력, np.argmax는 tf.argmax와 같은 기능
    #결괏값인 labels의 i번째 요소가 원-핫 인코딩 형식으로 되어 있으므로, 해당 배열에가 가장 높은 값을 가진 인덱스를 예측한 숫자로 출력
    # 1차원 배열로 되어 있는 i번째 이미지 데이터를 28x28 형식의 2차원 배열로 변형하여 이미지 형태로 출력
    # cmap 파라미터를 통해 이미지를 그레이 스케일로 출력
    subplot.imshow(mnist.test.images[i].reshape((28,28)), cmap=plt.cm.gray_r)

In [19]:
plt.show()